<a href="https://colab.research.google.com/github/RukadeAkshay01/hello_world/blob/main/ziroeda_qwen_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Fine-tune the Qwen model specified by the `model_id` variable using the `train_ready.jsonl` dataset, then save the trained model.

## Install Libraries

### Subtask:
Install all necessary Python libraries required for training the Qwen model, such as `transformers`, `torch`, `datasets`, etc.


**Reasoning**:
To fulfill the subtask of installing necessary Python libraries, I will use `pip install` within a code block to install `transformers`, `accelerate`, `peft`, `bitsandbytes`, `datasets`, and `tqdm`.



In [10]:
pip install transformers accelerate peft bitsandbytes datasets tqdm

## Load Training Data

### Subtask:
Load the `train_ready.jsonl` file into a suitable format (e.g., a pandas DataFrame or a Hugging Face Dataset object).


**Reasoning**:
I need to load the `train_ready.jsonl` file into a Hugging Face Dataset object. The first step is to import the `load_dataset` function from the `datasets` library.



In [11]:
from datasets import load_dataset

print("Imported load_dataset from the datasets library.")

Imported load_dataset from the datasets library.


**Reasoning**:
Now that `load_dataset` is imported, I will use it to load the `train_ready.jsonl` file, specifying 'json' as the path and the file name as the `data_files` argument, storing the result in a variable named `dataset`.



In [12]:
dataset = load_dataset('json', data_files='/content/train_ready.jsonl')
print(f"Dataset loaded: {dataset}")

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 145
    })
})


## Load Qwen 2.5 Instruct Model and Tokenizer

### Subtask:
Initialize the Qwen 2.5 Instruct model and its corresponding tokenizer from the `transformers` library using the `model_id` variable set in the kernel state.


**Reasoning**:
To initialize the Qwen 2.5 Instruct model and its tokenizer, I need to import the necessary classes from the `transformers` library, specifically `AutoTokenizer` and `AutoModelForCausalLM`.



In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Imported AutoTokenizer and AutoModelForCausalLM from the transformers library.")

Imported AutoTokenizer and AutoModelForCausalLM from the transformers library.


**Reasoning**:
Now that the necessary classes are imported, I will load the tokenizer and model using `AutoTokenizer.from_pretrained()` and `AutoModelForCausalLM.from_pretrained()` respectively, passing the `model_id` and setting `device_map='auto'` for the model, then print a confirmation.



In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto'
)

print(f"Tokenizer and model for {model_id} loaded successfully.")

Tokenizer and model for Qwen/Qwen2-1.5B-Instruct loaded successfully.


## Prepare Dataset for Training

### Subtask:
Preprocess the loaded data by tokenizing it and formatting it into a dataset object suitable for training with the Qwen model.


**Reasoning**:
To preprocess the data, I will first set the `pad_token` for the tokenizer to its `eos_token` as required for causal language models, and then define a tokenization function to apply the chat template and tokenize the messages, preparing the dataset for training.



In [15]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    # Apply chat template and tokenize messages
    tokenized_output = tokenizer.apply_chat_template(example["messages"], tokenize=True, truncation=True)
    return {"input_ids": tokenized_output}

# Apply the tokenization function to the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["messages"]["train"])

print("Tokenizer's pad token set to eos token.")
print("Dataset tokenized successfully.")
print("\nFirst tokenized example:")
print(tokenized_dataset[0])

<>:9: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:9: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/tmp/ipython-input-3578412688.py:9: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["messages"]["train"])


TypeError: list indices must be integers or slices, not str

**Reasoning**:
The previous code failed because the `remove_columns` argument was incorrectly trying to access `["messages"]["train"]` which resulted in a `TypeError`. The `remove_columns` argument expects a list of column names (strings) to be removed from the dataset after mapping. The correct way to specify the column to remove is `["messages"]`.



In [16]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    # Apply chat template and tokenize messages
    tokenized_output = tokenizer.apply_chat_template(example["messages"], tokenize=True, truncation=True)
    return {"input_ids": tokenized_output}

# Apply the tokenization function to the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["messages"])

print("Tokenizer's pad token set to eos token.")
print("Dataset tokenized successfully.")
print("\nFirst tokenized example:")
print(tokenized_dataset["train"][0])

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Tokenizer's pad token set to eos token.
Dataset tokenized successfully.

First tokenized example:
{'input_ids': [151644, 8948, 198, 2610, 525, 1863, 8698, 86761, 11, 458, 15235, 6203, 304, 20185, 27218, 11708, 66147, 13, 4615, 16538, 374, 311, 6923, 4240, 11, 364, 2271, 35258, 6, 1707, 49054, 33340, 369, 3738, 91494, 323, 19038, 13, 31399, 26310, 19819, 3692, 49184, 320, 68, 1302, 2572, 46281, 389, 2115, 11, 58414, 389, 1290, 8, 323, 5978, 279, 6789, 5868, 16645, 389, 264, 4108, 10041, 13, 151645, 198, 151644, 872, 198, 43504, 2801, 16799, 952, 7515, 921, 785, 16224, 11574, 264, 95932, 8872, 1667, 458, 18117, 18, 17, 323, 458, 362, 19, 24, 23, 23, 95932, 5579, 13, 576, 18117, 18, 17, 21308, 2524, 17289, 311, 279, 362, 19, 24, 23, 23, 11, 892, 304, 2484, 20118, 279, 95932, 8872, 382, 58, 23707, 26069, 50, 921, 12, 3034, 25, 16541, 760, 3990, 25, 4479, 12, 24610, 18, 17, 25758, 8226, 1786, 8273, 19, 198, 12, 3034, 25, 68770, 16, 760, 3990, 25, 264, 19, 24, 23, 23, 198, 12, 3034, 25, 9593

## Configure Training Arguments

### Subtask:
Define the training arguments, including parameters like learning rate, batch size, number of epochs, and output directory.


**Reasoning**:
To configure the training arguments, I need to import the `TrainingArguments` class from the `transformers` library, define an output directory, and then instantiate `TrainingArguments` with the specified parameters.



In [17]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    tf32=True
)

print(f"Training arguments configured. Output directory: {output_dir}")

ValueError: --tf32 requires Ampere or a newer GPU arch, cuda>=11 and torch>=1.7

**Reasoning**:
The previous code failed because the `tf32=True` argument in `TrainingArguments` requires specific GPU hardware and software versions that are not met in the current environment. To resolve this, I will remove the `tf32=True` argument from the `TrainingArguments` instantiation.



In [18]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True
)

print(f"Training arguments configured. Output directory: {output_dir}")

Training arguments configured. Output directory: ./qwen_finetuned


**Reasoning**:
With the training arguments now correctly configured, the next logical step is to define a data collator. This collator will be responsible for dynamically padding the tokenized input sequences to the maximum length within each batch and for creating the labels required for language modeling, ensuring the data is in the correct format for training.



In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("Data collator for language modeling initialized.")

Data collator for language modeling initialized.


## Initialize and Train Model

### Subtask:
Initialize the Hugging Face `Trainer` with the Qwen model, tokenizer, training arguments, and prepared dataset, then start the training process.


**Reasoning**:
To initialize and train the model, I will import the `Trainer` class, instantiate it with the model, tokenizer, training arguments, tokenized dataset, and data collator, and then call its `train()` method to start the fine-tuning process.



In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-2972235760.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer initialized. Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


OutOfMemoryError: CUDA out of memory. Tried to allocate 52.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 46.12 MiB is free. Process 2493 has 14.69 GiB memory in use. Of the allocated memory 14.31 GiB is allocated by PyTorch, and 261.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

**Reasoning**:
The previous code failed with an `OutOfMemoryError`, indicating that the model training consumed too much GPU memory. To resolve this, I will enable gradient checkpointing in the model configuration to reduce memory usage during training and ensure the model is loaded with 4-bit quantization using `BitsAndBytesConfig` for further memory optimization.



In [21]:
from transformers import Trainer, BitsAndBytesConfig
import torch

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized. Starting training...")
trainer.train()
print("Training complete.")

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

**Reasoning**:
The previous code failed because the `device_map='auto'` argument in `from_pretrained`, when used with `quantization_config`, sometimes tries to dispatch modules to the CPU, leading to a `ValueError`. To ensure the model fully leverages the GPU after 4-bit quantization, I will explicitly set `device_map='cuda'` to force all layers onto the GPU and also apply `prepare_model_for_kbit_training` for optimal training of quantized models.



In [22]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization, enable gradient checkpointing, and prepare for kbit training
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='cuda' # Explicitly set to 'cuda' to ensure all quantized modules are on GPU
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Re-initialize the Trainer with the modified model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized. Starting training...")
trainer.train()
print("Training complete.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 446.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 278.12 MiB is free. Process 2493 has 14.47 GiB memory in use. Of the allocated memory 14.09 GiB is allocated by PyTorch, and 255.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

**Reasoning**:
The previous code still resulted in an `OutOfMemoryError` even after 4-bit quantization and explicit `device_map='cuda'`, indicating that the model's memory footprint is still too large for direct fine-tuning. To address this, I will implement Parameter-Efficient Fine-Tuning (PEFT) using LoRA (Low-Rank Adaptation) by importing `LoraConfig` and `get_peft_model`, configuring LoRA with suitable parameters, and then applying it to the model before re-initializing the `Trainer` and attempting training again.



In [23]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='cuda' # Explicitly set to 'cuda' to ensure all quantized modules are on GPU
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8, # LoRA attention dimension
    lora_alpha=16, # Alpha parameter for LoRA scaling
    lora_dropout=0.1, # Dropout probability for LoRA layers
    bias="none", # Bias type for LoRA. Can be 'none', 'all', or 'lora_only'
    task_type="CAUSAL_LM", # Task type for the model
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized with PEFT model. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-1583867049.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer initialized with PEFT model. Starting training...


TypeError: device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index = -1)


**Reasoning**:
The previous `TypeError` related to `device()` with a `NoneType` argument suggests an issue with how `accelerate` (used by `Trainer`) interacts with explicit `device_map='cuda'` when combined with quantization and PEFT. To allow `accelerate` to manage device placement more effectively, I will revert `device_map='cuda'` to `device_map='auto'` in the `from_pretrained` call. With PEFT and gradient checkpointing enabled, the model's memory footprint should be sufficiently reduced to prevent `OutOfMemoryError` and enable `device_map='auto'` to keep all necessary modules on the GPU.



In [24]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto' # Changed from 'cuda' to 'auto' to let accelerate handle device placement
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8, # LoRA attention dimension
    lora_alpha=16, # Alpha parameter for LoRA scaling
    lora_dropout=0.1, # Dropout probability for LoRA layers
    bias="none", # Bias type for LoRA. Can be 'none', 'all', or 'lora_only'
    task_type="CAUSAL_LM", # Task type for the model
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized with PEFT model. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3280238529.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer initialized with PEFT model. Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.69 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.33 GiB is free. Process 2493 has 11.41 GiB memory in use. Of the allocated memory 7.46 GiB is allocated by PyTorch, and 3.82 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

**Reasoning**:
The training process is still encountering an `OutOfMemoryError` even after implementing 4-bit quantization, gradient checkpointing, and PEFT (LoRA). This indicates that the current batch size is still too large for the available GPU memory. To further reduce memory consumption, I will decrease the `per_device_train_batch_size` from 2 to 1 in the `TrainingArguments`, which will halve the memory footprint per processing step while maintaining the same effective batch size by keeping `gradient_accumulation_steps` at 4.



In [25]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True
)

print(f"Training arguments configured. Output directory: {output_dir}")

Training arguments configured. Output directory: ./qwen_finetuned


**Reasoning**:
With the `TrainingArguments` updated to reduce memory consumption, I will now re-initialize the `Trainer` with the PEFT-adapted model and the new training arguments. Then, I will call `trainer.train()` to attempt the fine-tuning process again, expecting it to complete successfully without `OutOfMemoryError`.



In [26]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model and updated training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model and updated training arguments. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3571423482.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer re-initialized with PEFT model and updated training arguments. Starting training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.85 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.70 GiB is free. Process 2493 has 13.04 GiB memory in use. Of the allocated memory 10.25 GiB is allocated by PyTorch, and 2.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

**Reasoning**:
The training process continues to encounter an `OutOfMemoryError` even after implementing 4-bit quantization, gradient checkpointing, LoRA, and reducing the `per_device_train_batch_size`. This suggests that further memory optimization is needed. To address this, I will re-configure the `TrainingArguments` by adding `group_by_length=True` to minimize padding and `torch_compile=True` to leverage PyTorch's compiler for potential memory and speed improvements. Then, I will re-initialize the Trainer and retry the training process.



In [27]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    torch_compile=True
)

print(f"Training arguments configured. Output directory: {output_dir}")

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.


Training arguments configured. Output directory: ./qwen_finetuned


**Reasoning**:
With the `TrainingArguments` now updated with `group_by_length=True` and `torch_compile=True`, the next step is to re-initialize the `Trainer` with the PEFT-adapted model and these new training arguments. Then, I will call `trainer.train()` to attempt the fine-tuning process again, expecting it to complete successfully without `OutOfMemoryError` due to these additional memory optimizations.



In [28]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model and updated training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model and updated training arguments. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3571423482.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer re-initialized with PEFT model and updated training arguments. Starting training...


/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
W1211 13:26:55.266000 196 torch/_inductor/utils.py:1558] [18/0] Not enough SMs to use max_autotune_gemm mode


BackendCompilerFailed: backend='inductor' raised:
OutOfMemoryError: CUDA out of memory. Tried to allocate 2.45 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.70 GiB is free. Process 2493 has 13.04 GiB memory in use. Of the allocated memory 9.14 GiB is allocated by PyTorch, and 3.77 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


**Reasoning**:
The previous code failed with an `OutOfMemoryError` during compilation by `torch_compile` (backend='inductor'), indicating that this optimization itself is consuming too much GPU memory. To resolve this, I will disable `torch_compile` by setting `torch_compile=False` in the `TrainingArguments` to prevent memory issues during the compilation phase, while retaining other memory-saving techniques.



In [29]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    torch_compile=False
)

print(f"Training arguments configured. Output directory: {output_dir}")

Training arguments configured. Output directory: ./qwen_finetuned


**Reasoning**:
With `torch_compile` disabled in the `TrainingArguments`, I will now re-initialize the `Trainer` with the PEFT-adapted model and these new training arguments. Then, I will call `trainer.train()` to attempt the fine-tuning process again, expecting it to complete successfully without `OutOfMemoryError`.



In [30]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model and updated training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model and updated training arguments. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3571423482.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer re-initialized with PEFT model and updated training arguments. Starting training...


BackendCompilerFailed: backend='inductor' raised:
OutOfMemoryError: CUDA out of memory. Tried to allocate 2.45 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.69 GiB is free. Process 2493 has 13.05 GiB memory in use. Of the allocated memory 10.22 GiB is allocated by PyTorch, and 2.70 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


**Reasoning**:
The training process continues to fail with `BackendCompilerFailed` (an `OutOfMemoryError` originating from `torch._inductor`), even after explicitly setting `torch_compile=False` in `TrainingArguments` and applying all other memory optimizations like 4-bit quantization, gradient checkpointing, LoRA, and reduced batch size. This indicates that `torch.compile` is still being implicitly invoked or performing memory-intensive internal checks despite the `TrainingArguments` flag. To definitively prevent `torch.compile` from interfering, I will patch `torch.compile` to a no-op function before re-initializing the model and trainer.



In [31]:
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import functools

# Patch torch.compile to be a no-op function
_original_torch_compile = torch.compile
def _noop_torch_compile(model, *args, **kwargs):
    # print("torch.compile was called, but is being bypassed.")
    return model
torch.compile = _noop_torch_compile

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Re-initialize the Trainer with the PEFT-adapted model and updated training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model and updated training arguments. Starting training...")
trainer.train()
print("Training complete.")

# Restore original torch.compile after training (optional, but good practice)
torch.compile = _original_torch_compile


/tmp/ipython-input-1926541051.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer re-initialized with PEFT model and updated training arguments. Starting training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.36 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.61 GiB is free. Process 2493 has 13.13 GiB memory in use. Of the allocated memory 10.41 GiB is allocated by PyTorch, and 2.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Task
The training process continues to encounter an `OutOfMemoryError`, despite applying 4-bit quantization, gradient checkpointing, LoRA, a reduced `per_device_train_batch_size=1`, and disabling `torch_compile`. This persistence indicates that the model's memory footprint, even with these optimizations, still exceeds the available GPU memory. A significant contributing factor to memory usage is the length of the input sequences. While `truncation=True` was used, an explicit `max_seq_length` was not defined, meaning sequences could still be very long and consume substantial memory.

To resolve this, I will implement the following steps:
1.  **Clear GPU memory**: Explicitly delete the current `model` and `tokenizer` instances and clear the CUDA cache to free up all possible GPU memory.
2.  **Re-initialize tokenizer and model with `max_seq_length`**:
    *   Re-initialize the Qwen 2.5 Instruct tokenizer.
    *   Define a `max_seq_length` (e.g., 1024) to limit the length of tokenized sequences.
    *   Modify the `tokenize_function` to include this `max_seq_length` for truncation.
    *   Re-tokenize the dataset using the new `max_seq_length`.
    *   Re-initialize the Qwen 2.5 Instruct model with 4-bit quantization, `device_map='auto'`, `gradient_checkpointing_enable()`, and `prepare_model_for_kbit_training()`.
    *   Re-apply LoRA configuration (`lora_config`) to the model using `get_peft_model()`.
3.  **Configure Training Arguments**: Re-use the previously configured `training_args` (which include `per_device_train_batch_size=1`, `gradient_accumulation_steps=4`, `fp16=True`, `group_by_length=True`, and `torch_compile=False`).
4.  **Initialize and Train Model**: Instantiate the `Trainer` with the newly re-initialized and optimized model, the re-tokenized dataset, and the `data_collator`. Then, attempt to train the model.
5.  **Save Trained Model**: If training completes successfully, save the fine-tuned Qwen model and tokenizer to the specified output directory.

This approach focuses on directly reducing the memory footprint of the input data by controlling sequence length, in addition to all previously applied memory optimizations.

```python
import torch
import gc
from transformers import Trainer, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset # Assuming dataset is already loaded

# Step 1: Clear GPU memory and reset variables from previous attempts
del model
del tokenizer
del trainer
gc.collect()
torch.cuda.empty_cache()

# Patch torch.compile to be a no-op function for the current session (if still active)
import functools
_original_torch_compile = torch.compile
def _noop_torch_compile(model, *args, **kwargs):
    print("torch.compile was called, but is being bypassed.")
    return model
torch.compile = _noop_torch_compile

print("Cleared GPU memory and reset model/tokenizer. torch.compile patched to no-op.")

# Define max_seq_length
max_seq_length = 1024 # You might need to adjust this based on GPU memory

# Step 2: Re-initialize tokenizer and model with max_seq_length
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # Ensure pad_token is set

def tokenize_function_with_max_length(example):
    # Apply chat template and tokenize messages with explicit max_seq_length
    tokenized_output = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        truncation=True,
        max_length=max_seq_length
    )
    return {"input_ids": tokenized_output, "labels": tokenized_output.copy()} # Labels for Causal LM

# Re-tokenize the dataset with the new max_seq_length
tokenized_dataset = dataset.map(tokenize_function_with_max_length, batched=True, remove_columns=["messages"])

print(f"Dataset re-tokenized with max_seq_length: {max_seq_length}")
print("\nFirst re-tokenized example:")
print(tokenized_dataset["train"][0])

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

print("Model re-initialized with 4-bit quantization, gradient checkpointing, and LoRA.")

# Step 3: Configure Training Arguments (re-using previous successful configuration)
output_dir = "./qwen_finetuned" # Ensure this variable is defined in the kernel state

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1, # Already set to 1
    gradient_accumulation_steps=4, # Already set to 4
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True, # Already set to True
    torch_compile=False # Already set to False
)

print(f"Training arguments configured. Output directory: {output_dir}")

# Initialize DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Step 4: Initialize and Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer, # Keep for compatibility, processing_class is preferred in newer versions
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model, updated training arguments, and re-tokenized dataset. Starting training...")
trainer.train()
print("Training complete.")

# Restore original torch.compile after training (good practice)
torch.compile = _original_torch_compile

# Step 5: Save Trained Model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Fine-tuned model and tokenizer saved to {output_dir}")

# Step 6: Final Task: Confirm that the Qwen model has been successfully trained and saved.
print("Qwen model successfully trained and saved.")
```

## Explain Persistent OutOfMemoryError

### Subtask:
Analyze the reasons for the persistent OutOfMemoryError despite previous optimizations.


The persistent `OutOfMemoryError` encountered during training, despite implementing 4-bit quantization, gradient checkpointing, LoRA, reduced batch size (`per_device_train_batch_size=1`), and attempting to disable `torch_compile`, indicates a fundamental issue with the memory footprint of the input data itself, specifically related to the length of the tokenized sequences.

### Key Contributing Factor: Unbounded Input Sequence Length

In the `tokenize_function` step, the `tokenizer.apply_chat_template` was called with `truncation=True` but without explicitly setting a `max_length`. While `truncation=True` ensures that sequences exceeding the tokenizer's model maximum length (if defined) are truncated, it doesn't enforce a specific, smaller `max_length` that is suitable for the available GPU memory. The default `max_length` for many large language models can still be quite substantial (e.g., 2048, 4096, or even higher for Qwen models), especially for an instruct model that might handle complex multi-turn conversations.

When `max_length` is not explicitly set, the tokenizer defaults to the model's maximum sequence length (if it exists) or a very large number, leading to:

1.  **Large `input_ids` tensors**: Even with `per_device_train_batch_size=1`, if a single tokenized input sequence is very long, the resulting tensor of `input_ids` and its corresponding attention mask can consume a significant amount of GPU memory.
2.  **Activations during gradient checkpointing**: While gradient checkpointing reduces memory by not storing intermediate activations for the entire backward pass, it still recomputes parts of them. The memory required for these recomputations, especially for very long sequences, can still exceed available GPU memory, particularly for deeper layers.
3.  **PEFT (LoRA) overhead**: Although PEFT significantly reduces the number of trainable parameters and thus the optimizer states' memory footprint, the base model activations for long sequences are still processed and contribute to memory usage.

In summary, the `OutOfMemoryError` persists because, despite all other optimizations, the maximum length of the tokenized sequences was not explicitly constrained to a value that fits within the GPU's memory limits, causing individual input tensors to be too large during forward and backward passes.

## Clear GPU Memory and Re-initialize Model with Further Optimizations

### Subtask:
Explicitly delete existing model and tokenizer instances, clear CUDA cache, re-initialize the Qwen 2.5 Instruct model with 4-bit quantization, gradient checkpointing, LoRA, and define a smaller `max_seq_length` for tokenization, applying it when preparing the dataset.


**Reasoning**:
To free up GPU memory and ensure a clean slate before re-initializing the model with new configurations, I will explicitly delete the `model` and `trainer` objects, then clear the CUDA cache. I will also patch `torch.compile` to a no-op function as it was causing issues previously, making sure it doesn't interfere with subsequent steps.



In [32]:
import gc

# Delete existing model and trainer instances to free up memory
del model
del trainer

# Clear CUDA cache
torch.cuda.empty_cache()
gc.collect()

# Patch torch.compile to be a no-op function as it caused OOM in previous attempts
_original_torch_compile = torch.compile
def _noop_torch_compile(model, *args, **kwargs):
    return model
torch.compile = _noop_torch_compile

print("GPU memory cleared, model and trainer deleted, and torch.compile patched.")

GPU memory cleared, model and trainer deleted, and torch.compile patched.


**Reasoning**:
Now that the GPU memory has been cleared and `torch.compile` patched, I will define `max_seq_length`, re-initialize the tokenizer with its `pad_token` set, and define a new `tokenize_function_with_max_length` to include this `max_seq_length` for truncation and label creation.



In [33]:
max_seq_length = 1024 # You might need to adjust this based on GPU memory

# Re-initialize the tokenizer from model_id
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # Ensure pad_token is set

def tokenize_function_with_max_length(example):
    # Apply chat template and tokenize messages with explicit max_seq_length
    tokenized_output = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        truncation=True,
        max_length=max_seq_length
    )
    return {"input_ids": tokenized_output, "labels": tokenized_output.copy()} # Labels for Causal LM

print(f"max_seq_length defined: {max_seq_length}")
print("Tokenizer re-initialized and tokenization function defined.")

max_seq_length defined: 1024
Tokenizer re-initialized and tokenization function defined.


**Reasoning**:
Now that `tokenize_function_with_max_length` is defined, I will apply it to the `dataset` using the `.map()` method with `batched=True` and `remove_columns=["messages"]` to re-tokenize the dataset with the specified `max_seq_length` and prepare it for training.



In [34]:
tokenized_dataset = dataset.map(tokenize_function_with_max_length, batched=True, remove_columns=["messages"])

print(f"Dataset re-tokenized with max_seq_length: {max_seq_length}")
print("\nFirst re-tokenized example:")
print(tokenized_dataset["train"][0])

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Dataset re-tokenized with max_seq_length: 1024

First re-tokenized example:
{'input_ids': [151644, 8948, 198, 2610, 525, 1863, 8698, 86761, 11, 458, 15235, 6203, 304, 20185, 27218, 11708, 66147, 13, 4615, 16538, 374, 311, 6923, 4240, 11, 364, 2271, 35258, 6, 1707, 49054, 33340, 369, 3738, 91494, 323, 19038, 13, 31399, 26310, 19819, 3692, 49184, 320, 68, 1302, 2572, 46281, 389, 2115, 11, 58414, 389, 1290, 8, 323, 5978, 279, 6789, 5868, 16645, 389, 264, 4108, 10041, 13, 151645, 198, 151644, 872, 198, 43504, 2801, 16799, 952, 7515, 921, 785, 16224, 11574, 264, 95932, 8872, 1667, 458, 18117, 18, 17, 323, 458, 362, 19, 24, 23, 23, 95932, 5579, 13, 576, 18117, 18, 17, 21308, 2524, 17289, 311, 279, 362, 19, 24, 23, 23, 11, 892, 304, 2484, 20118, 279, 95932, 8872, 382, 58, 23707, 26069, 50, 921, 12, 3034, 25, 16541, 760, 3990, 25, 4479, 12, 24610, 18, 17, 25758, 8226, 1786, 8273, 19, 198, 12, 3034, 25, 68770, 16, 760, 3990, 25, 264, 19, 24, 23, 23, 198, 12, 3034, 25, 95932, 16, 760, 3990, 25, 

**Reasoning**:
With the dataset re-tokenized using a defined `max_seq_length`, I will now re-define the `BitsAndBytesConfig`, load the Qwen model with 4-bit quantization, enable gradient checkpointing, prepare it for kbit training, and then apply the `LoraConfig` to the model. This step re-initializes the model with all the necessary memory optimizations to prevent `OutOfMemoryError`.



In [35]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure 4-bit quantization
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load the model with 4-bit quantization and enable gradient checkpointing
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bitsandbytes_config,
    device_map='auto'
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

print("Model re-initialized with 4-bit quantization, gradient checkpointing, and LoRA.")

Model re-initialized with 4-bit quantization, gradient checkpointing, and LoRA.


**Reasoning**:
Following the overall task instructions, I will re-configure the `TrainingArguments` using the last successful configuration which includes `per_device_train_batch_size=1`, `gradient_accumulation_steps=4`, `fp16=True`, `group_by_length=True`, and `torch_compile=False` to ensure optimal memory usage and prevent prior OOM errors.



In [36]:
from transformers import TrainingArguments

output_dir = "./qwen_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    torch_compile=False
)

print(f"Training arguments configured. Output directory: {output_dir}")

Training arguments configured. Output directory: ./qwen_finetuned


**Reasoning**:
Now that the training arguments are configured, I will initialize the `DataCollatorForLanguageModeling` using the tokenizer and then instantiate the `Trainer` with the newly re-initialized and optimized model, the re-tokenized dataset, and the training arguments. Finally, I will attempt to train the model.



In [37]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Initialize DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize and Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer re-initialized with PEFT model, updated training arguments, and re-tokenized dataset. Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3232299173.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Trainer re-initialized with PEFT model, updated training arguments, and re-tokenized dataset. Starting training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.364100
20,1.243200
30,1.197600
40,1.000400
50,0.788300
60,0.688600
70,0.697200
80,0.615200
90,0.650000
100,0.595800


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Training complete.


**Reasoning**:
Since the training process completed successfully, the next step is to save the fine-tuned model and its tokenizer to the specified output directory to fulfill the overall task requirement.



In [38]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Fine-tuned model and tokenizer saved to {output_dir}")

print("Qwen model successfully trained and saved.")

Fine-tuned model and tokenizer saved to ./qwen_finetuned
Qwen model successfully trained and saved.


## Save Trained Model

### Subtask:
Save the fine-tuned Qwen model and tokenizer to a specified directory.


**Reasoning**:
The training process has completed successfully, so the next logical step is to save the fine-tuned model and its tokenizer to the `output_dir` as instructed by the subtask.



In [39]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Fine-tuned model and tokenizer saved to {output_dir}")

Fine-tuned model and tokenizer saved to ./qwen_finetuned


## Final Task

### Subtask:
Confirm that the Qwen model has been successfully trained and saved.


## Summary:

### Q&A
**1. What was the primary reason for the persistent `OutOfMemoryError` despite applying numerous memory optimization techniques?**
The primary reason for the persistent `OutOfMemoryError` was the unbounded length of input sequences during tokenization. Although `truncation=True` was used, an explicit `max_seq_length` was not defined, leading to excessively long tokenized sequences that consumed too much GPU memory, even with a `per_device_train_batch_size=1`.

**2. Was the Qwen model successfully trained and saved?**
Yes, the Qwen model was successfully fine-tuned and saved to the `./qwen_finetuned` directory after implementing the corrective memory optimization strategies.

### Data Analysis Key Findings
*   The initial `OutOfMemoryError` was attributed to the lack of an explicit `max_seq_length` during tokenization, causing input sequences to be too large, even after applying 4-bit quantization, LoRA, gradient checkpointing, `per_device_train_batch_size=1`, and disabling `torch_compile`.
*   GPU memory was successfully cleared by deleting previous model/trainer instances and emptying the CUDA cache. `torch.compile` was patched to a no-op to prevent re-activation.
*   A `max_seq_length` of `1024` was defined and successfully applied during tokenization, drastically reducing the memory footprint of input sequences.
*   The Qwen model was re-initialized with 4-bit quantization (`bnb_4bit_quant_type="nf4"`, `bnb_4bit_compute_dtype=torch.float16`), gradient checkpointing enabled, and LoRA (`r=8`, `lora_alpha=16`) re-applied.
*   The training process completed successfully without `OutOfMemoryError` using the re-initialized model, re-tokenized dataset, and optimized training arguments (e.g., `num_train_epochs=3`, `per_device_train_batch_size=1`, `gradient_accumulation_steps=4`, `fp16=True`).
*   The fine-tuned Qwen model and its tokenizer were successfully saved to the `./qwen_finetuned` directory.

### Insights or Next Steps
*   For future large language model fine-tuning tasks, always explicitly define and experiment with `max_seq_length` to balance model performance and available GPU memory, even when other memory optimizations are in place.
*   Consider evaluating the fine-tuned model's performance on a validation set to ensure that the chosen `max_seq_length` and other training parameters have yielded a satisfactory balance between memory efficiency and model quality.


## Test the Fine-tuned Model

### Subtask:
Load the fine-tuned model and tokenizer and perform a text generation task to verify its functionality.

**Reasoning**:
To test the fine-tuned model, I will load the model and tokenizer from the `output_dir` (where they were saved), and then use the `pipeline` function from `transformers` for convenient text generation. I will also clear GPU memory before loading to ensure resources are available for inference.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gc

# Clear GPU memory before loading the model for inference
del model # Delete the Trainer's model instance
gc.collect()
torch.cuda.empty_cache()

print("GPU memory cleared for inference model loading.")

# Load the fine-tuned tokenizer and model
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(output_dir)
fine_tuned_model = AutoModelForCausalLM.from_pretrained(output_dir, device_map='auto', torch_dtype=torch.float16)

print(f"Fine-tuned model and tokenizer loaded from {output_dir}.")

# Create a text generation pipeline
pipeline_generator = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=fine_tuned_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

print("Text generation pipeline created.")

**Reasoning**:
Now that the text generation pipeline is ready, I will define a prompt and use the pipeline to generate a response from the fine-tuned Qwen model. This will demonstrate how the model can be used for inference.

In [ ]:
prompt = "What is the capital of France?" # You can change this prompt

# Generate text using the fine-tuned model
response = pipeline_generator(prompt, max_new_tokens=50, num_return_sequences=1)

print("Generated Response:")
print(response[0]["generated_text"])